In [18]:
# !pip install pymupdf

# !pip install pymupdf4llm

# !pip install pdfplumber
# !pip install pandas
# !pip install tabulate

In [40]:
!pip install -U llama-index --upgrade --no-cache-dir --force-reinstall
!pip install llama-parse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 2.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 8.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 10.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.8/961.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 10.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: striprtf
    Found existing installation: striprtf 0.0.26
    Uninstalling striprtf-0.0.26:
      Successfully uninstalled striprtf-0.0.26
 

In [ ]:
import fitz

# doc = fitz.open('/Users/choihyelynn/Documents/GitHub/pdf-preprocessing/아이엠뱅크 신탁/(고객용) 상세상품설명서 20241025(비케이씨에스제오차).pdf')
doc = fitz.open('/Users/choihyelynn/Documents/GitHub/pdf-preprocessing/가계대출상품설명서_23.01.16.pdf')

for page in doc:
    text = page.get_text('text')  # Retains layout
    print(text)

# blocks = page.get_text('blocks')
# for b in blocks:
#     print(b[4])  # Print the text content of each block


In [ ]:
# Using PyMuPDF4LLM

import subprocess
import os

input_directory = "/Users/choihyelynn/Documents/GitHub/pdf-preprocessing/trust_pdfs/"
output_directory = "/Users/choihyelynn/Documents/GitHub/pdf-preprocessing/trust_markdowns/"

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Command to run in the terminal
for filename in os.listdir(input_directory):
    if filename.endswith(".pdf"):  # Process only PDF files
        file_path = os.path.join(input_directory, filename)
        
        # Command to run markdowntext.py with the file as input
        command = ['python', 'markdowntext.py', file_path, output_directory]
        
        # Execute the command
        subprocess.run(command)

In [ ]:
import pdfplumber

with pdfplumber.open('/Users/choihyelynn/Documents/GitHub/pdf-preprocessing/가계대출상품설명서_23.01.16.pdf') as f:
    for page in f.pages:
        print(page.extract_text())
        print(page.extract_tables())

In [43]:
import pdfplumber
import pandas as pd
from pdfplumber.utils import extract_text, get_bbox_overlap, obj_to_bbox
import tabulate
import os

def process_pdf(pdf_path):
    pdf = pdfplumber.open(pdf_path)
    all_text = []

    #iterate over each page
    for page in pdf.pages:
        filtered_page = page  #current page
        chars = filtered_page.chars  #captures all characters on the filtered_page
    
        #iterate over each table found on the page
        for table in page.find_tables():

            first_table_char = page.crop(table.bbox).chars[0]  #stores the first character of the cropped table area
            filtered_page = filtered_page.filter(lambda obj: 
                get_bbox_overlap(obj_to_bbox(obj), table.bbox) is None
            )  # updated by filtering out characters that overlap with the table's bounding box
            chars = filtered_page.chars

            #create a DataFrame from the extracted table data
            df = pd.DataFrame(table.extract())
            df.columns = df.iloc[0]  #first row set as the header
            markdown = df.drop(0).to_markdown(index=False)  # rest converted to Markdown format
            chars.append(first_table_char  | {"text": markdown})  #first_table_char is updated with the markdown content and appended to chars
        
        page_text = extract_text(chars, layout = True) #extracts the text from the filtered characters with layout preservation
        all_text.append(page_text)
    
    pdf.close()
    return "\n".join(all_text)

file = r"/Users/choihyelynn/Documents/GitHub/pdf-preprocessing/example/가계대출상품설명서_23.01.16.pdf"
# file = r"/Users/choihyelynn/Documents/GitHub/pdf-preprocessing/아이엠뱅크 신탁/(고객용) 상세상품설명서 20241025(비케이씨에스제오차).pdf"
extracted_text = process_pdf(file)
print(extracted_text)

IndexError: list index out of range

In [47]:
import pdfplumber

with pdfplumber.open("/Users/choihyelynn/Documents/GitHub/pdf-preprocessing/trust_pdfs/(고객용) 상세상품설명서 20241025(비케이씨에스제오차).pdf") as pdf:
    # first_page = pdf.pages[0]
    # print(first_page.chars[0])
    # page_img = pdf_pages[0].to_image().original
    im = pdf.pages[0].to_image(resolution=300)
    im.show()

TypeError: 'type' object is not subscriptable

In [ ]:
import pdfplumber
import pandas as pd
from pdfplumber.utils import extract_text, get_bbox_overlap, obj_to_bbox
import os

def process_pdf(pdf_path):
    pdf = pdfplumber.open(pdf_path)
    all_text = []

    # Iterate over each page
    for page in pdf.pages:
        filtered_page = page  # current page
        chars = filtered_page.chars  # captures all characters on the filtered_page
    
        # Iterate over each table found on the page
        for table in page.find_tables():
            first_table_char = page.crop(table.bbox).chars[0]  # stores the first character of the cropped table area
            filtered_page = filtered_page.filter(lambda obj: 
                get_bbox_overlap(obj_to_bbox(obj), table.bbox) is None
            )  # updated by filtering out characters that overlap with the table's bounding box
            chars = filtered_page.chars

            # Create a DataFrame from the extracted table data
            df = pd.DataFrame(table.extract())
            df.columns = df.iloc[0]  # first row set as the header
            markdown = df.drop(0).to_markdown(index=False)  # rest converted to Markdown format
            chars.append(first_table_char | {"text": markdown})  # Append the table markdown to the characters
        
        page_text = extract_text(chars, layout=True)  # Extracts the text from the filtered characters with layout preservation
        all_text.append(page_text)
    
    pdf.close()
    return "\n".join(all_text)

# Define input and output directories
input_directory = "/Users/choihyelynn/Documents/GitHub/pdf-preprocessing/trust_pdfs/"
output_directory = "/Users/choihyelynn/Documents/GitHub/pdf-preprocessing/trust_txts/"

# Create output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Process all PDF files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith(".pdf"):
        file_path = os.path.join(input_directory, filename)
        extracted_text = process_pdf(file_path)

        # Define output text file name
        output_file = os.path.join(output_directory, filename.replace(".pdf", ".txt"))

        # Save the extracted text to a .txt file
        with open(output_file, "w", encoding="utf-8") as text_file:
            text_file.write(extracted_text)

        print(f"Processed and saved: {output_file}")


In [ ]:
page=doc.load_page(0)
print(page.get_text())